 # Tratar o .xlsx para um csv, fazendo requisições para os links das matérias


In [1]:
!pip install openpyxl
import pandas as pd
import os
from bs4 import BeautifulSoup

In [2]:
arquivo_csv = '../data/central_de_fatos.csv'
# Leitura do arquivo CSV
df_csv = pd.read_csv(arquivo_csv, sep=';', encoding='utf-8')

In [3]:
arquivo_xlsx = '../data/desinfopedia_final.xlsx'

# Leitura do arquivo Excel
df_xlsx = pd.read_excel(arquivo_xlsx)
#df_xlsx = df_xlsx.drop_duplicates()
#print("\nArquivo XLSX sem duplicatas:")
df_xlsx.head()

,Título:,Link:,Fonte:,Data:
0,Maiara e Maraisa não gritaram ‘fora Bolsonaro’...,https://www.aosfatos.org/noticias/maiara-e-mar...,Aos Fatos,2022-09-16
1,"Lula, Bolsonaro, Ciro e Tebet erram ao falar d...",https://www.aosfatos.org/noticias/lula-bolsona...,Aos Fatos,2022-09-16
2,Vídeo que mostra Lula se entregando à PF em 20...,https://www.aosfatos.org/noticias/video-mostra...,Aos Fatos,2022-09-14
3,Anúncio de Thronicke engana ao dizer que uso d...,https://www.aosfatos.org/noticias/anuncio-de-t...,Aos Fatos,2022-09-14
4,Luís Roberto Barroso não é sogro de dono da Re...,https://www.aosfatos.org/noticias/barroso-stf-...,Aos Fatos,2022-09-14


In [4]:
import requests

# Renomear colunas do XLSX para coincidir com as do CSV
df_xlsx = df_xlsx.rename(columns={
    'Título:': 'title',
    'Link:': 'url',
    'Fonte:': 'source_name',
    'Data:': 'publication_date'
})

def extrair_primeiro_paragrafo(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            div = soup.find('div', id='entry-content')
            if div: #aosFatos
                p = div.find('p')
                #print(p.get_text(strip=True))
                return p.get_text(strip=True) if p else None
            else:
                if soup.find('p', class_=lambda c: c and 'content-text__container' in c): #G1
                    p = soup.find('p', class_=lambda c: c and 'content-text__container' in c)
                    return p.get_text(strip=True) if p else None
                #o site a lupa não tem um site bem caracterizado para extrair informações

        return None
    except Exception as e:
        print(f"Erro ao acessar {url}: {e}")
        return None

# Adicionar colunas ausentes ao XLSX para igualar ao CSV (com NaN)
for col in df_csv.columns:
    if col not in df_xlsx.columns:
        df_xlsx[col] = None

df_xlsx['rating'] = ['Falso'] * df_xlsx.shape[0]
# Aplica a função em cada linha da coluna 'url'
df_xlsx['text_news'] = df_xlsx['url'].apply(extrair_primeiro_paragrafo)
df_xlsx.to_csv('../data/desinfopedia_final_completo.csv', sep=';', index=False, encoding='utf-8-sig')

Erro ao acessar Música a favor de Lula é inserida em vídeo que mostra torcedores do Liverpool: Failed to parse: Música a favor de Lula é inserida em vídeo que mostra torcedores do Liverpool
Erro ao acessar Preço do arroz não subiu por causa de demarcação de terra indígena: Failed to parse: Preço do arroz não subiu por causa de demarcação de terra indígena
Erro ao acessar One7 não administra carreira de Gusttavo Lima | Aos Fatos: Failed to parse: One7 não administra carreira de Gusttavo Lima | Aos Fatos
Erro ao acessar CNN não disse que comandante russo acordou morto na Ucrânia | Aos Fatos: Failed to parse: CNN não disse que comandante russo acordou morto na Ucrânia | Aos Fatos
Erro ao acessar Vídeo em que Gusttavo Lima beija bandeira é antes do Lolla | Aos Fatos: Failed to parse: Vídeo em que Gusttavo Lima beija bandeira é antes do Lolla | Aos Fatos
Erro ao acessar PF não fez exposição de ‘dinheiro roubado' pelo PT | Aos Fatos: Failed to parse: PF não fez exposição de ‘dinheiro roubado